# OUTLINE
* Introduce DALBAR study, criticisms
* Methodology for using ETFs instead of MFs.  Reasoning - precise, daily, ascendant
* Choose a basket of ETFs: SPY, sectors, vol & other hot fads, small, trader funds (2x and 3x) etc...
* Pick one major fund in depth, track across time, 
* calculate a composite of many funds, find consistent level.  This is headline takeaway
* try comparing Vanguard to Powershares, etc...  Generalize
* try comparing boring funds to trendy funds. Generalize


   

# Timing of ETF Inflows and Outflows


## Introduction

...



In [368]:
from __future__ import print_function
from __future__ import division
from importlib import reload

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import seaborn as sns
import datetime

import os
os.getcwd()

## NOTE: Add paths to dependencies on any machine being used here.  Invalid paths will be silently ignored  
import sys
sys.path.append('/anaconda2/') # either place your config.py in this path, or add another sys.path.append() pointing to your path for config.py
import config ## TODO: place file called config.py in the /anaconda2/ folder

sys.path.append(config.REPO_ROOT+'data/') #Chad: Home
from prices.eod import iqfeed
from prices.eod.read import get_adjustment_factors




In [416]:
reload(iqfeed)
symbols = ['IBB','SPY','IWM','MUB']

#price = iqfeed.read_etf_daily(symbol)
price = iqfeed.read_etf_daily(symbols)


so = iqfeed.read_etf_so(symbols)
df = price[['close']].join(so)
df = df[df.so>0]

adj_factors = get_adjustment_factors(symbols).stack()
adj_factors.name = 'adj_factors'
#df = df.join(adj_factors)

df['close_adj'] = df.close * adj_factors

df['daily_ret'] = df.close.pct_change()
df['daily_ret_adj'] = df.close_adj.pct_change()

df['flow'] = df.so.pct_change()
df['mkt_cap'] = df.so*df.close
df['mkt_cap_adj'] = df.so*df.close_adj

df = df[df.daily_ret.abs()<0.2] # filter likely outliers

#df.dropna().tail()
#df.groupby(level='date').sum().resample('BM').sum().flow.abs().hist(bins=50)

In [442]:
def calc_twr(df,div_adj=True):
    if div_adj == True:
        cum_ret = (df.daily_ret_adj+1).product() - 1
    else:    
        cum_ret = (df.daily_ret+1).product() - 1
    return cum_ret

def calc_approx_mwr(df,div_adj=True):
    ''' Calculate the return for the full time period normalized for amount of capital deployed.
    '''
    if div_adj == True:
        # First, calculate the invested "basis" in terms of day 0 dollars.      
        adj_flows = df.flow*(df.mkt_cap_adj/df.mkt_cap_adj.iloc[0]) 
        adj_flows.iloc[0] = 1 # initial principal
        basis = (adj_flows).cumsum() #cumulative contributions in beginning of period dollars

        # Next, convert daily returns to log basis and weight by amount of capital deployed
        # to calculate an average compound return rate (log)
        log_rets = np.log(df.daily_ret_adj+1)
        
    else:
        # First, calculate the invested "basis" in terms of day 0 dollars.      
        adj_flows = df.flow*(df.mkt_cap/df.mkt_cap.iloc[0]) 
        adj_flows.iloc[0] = 1 # initial principal
        basis = (adj_flows).cumsum() #cumulative contributions in beginning of period dollars

        # Next, convert daily returns to log basis and weight by amount of capital deployed
        # to calculate an average compound return rate (log)
        log_rets = np.log(df.daily_ret+1)
        
    avg_daily_log_ret = (log_rets*basis / basis.mean()).mean()
    
    # finally, convert into a simple return for the full period
    cum_ret = (avg_daily_log_ret+1)**(len(log_rets))-1
    cr2 = np.exp(avg_daily_log_ret*len(basis))-1
    print(cr2)
    return cum_ret

df.sort_index(inplace=True)

for symbol in symbols:
    #tmp = df.xs(slice('2014-01-01','2016-12-31'),level='date',drop_level=False).xs(symbol,level='symbol',drop_level=True)
    tmp = df.xs(slice('2010-01-01','2016-12-31'),level='date',drop_level=False).xs(symbol,level='symbol',drop_level=True)
    print("For: {}".format(symbol))
    print("     MWR (adj): {}".format(calc_approx_mwr(tmp,True)))
    print("     TWR (adj): {}".format(calc_twr(tmp,True)))
    print("     MWR: {}".format(calc_approx_mwr(tmp,False)))
    print("     TWR: {}".format(calc_twr(tmp,False)))
    print()


For: IBB
3.0349885244299113
     MWR (adj): 3.0321709076138186
     TWR (adj): 3.216610800358125
2.9905209975875424
     MWR: 2.987778498692616
     TWR: 3.160047619891558

For: SPY
1.3197684764687496
     MWR (adj): 1.3192277142135018
     TWR (adj): 1.3145393690696014
0.9872106495894284
     MWR: 0.9869020808005136
     TWR: 1.0058327351041005

For: IWM
41.58689762851217
     MWR (adj): 41.39038490765354
     TWR (adj): 1.3952622622547683
26.617560018331925
     MWR: 26.517778319575704
     TWR: 1.1596732863549022

For: MUB
0.2458619885885378
     MWR (adj): 0.24584216019967942
     TWR (adj): 0.2875136873788646
-0.007986569484166428
     MWR: -0.00798659049377437
     TWR: 0.05284671532846419



In [428]:
def compare_annual(df):
    symbols = df.index.get_level_values('symbol').unique().tolist()
    
    out = pd.DataFrame()
    for symbol in symbols:
        twr = df.xs(symbol,level='symbol').resample('A').apply(calc_twr)
        twr.name = 'twr'
        mwr = df.xs(symbol,level='symbol').resample('A').apply(calc_approx_mwr)
        mwr.name = 'mwr'
        both = pd.concat([twr,mwr],axis=1).reset_index()
        both['symbol'] = symbol
        both.set_index(['date','symbol'],inplace=True)
        both['timing_impact'] = both.mwr - both.twr
        out = pd.concat([out,both],axis=0)
    return out

compare_annual(df).timing_impact.unstack()
#.xs(slice('2009-01-01','2009-12-31'),level='date',drop_level=False)

symbol,IBB,IWM,MUB,SPY
date,,,,
2001-12-31,NaN,0.040870,NaN,0.008642
2002-12-31,NaN,-0.035075,NaN,0.021983
2003-12-31,NaN,0.009124,NaN,-0.001100
2004-12-31,NaN,0.014258,NaN,0.038767
2005-12-31,NaN,0.070443,NaN,-0.000278
2006-12-31,NaN,-0.040335,NaN,0.009887
2007-12-31,NaN,-0.101906,0.001694,-0.006682
2008-12-31,-0.007705,-0.089825,0.015259,-0.024945
2009-12-31,-0.003937,-0.006195,0.205986,0.091894


In [429]:
df.loc['2015'].xs('IWM',level='symbol')

,close,so,close_adj,daily_ret,daily_ret_adj,flow,mkt_cap,mkt_cap_adj
date,,,,,,,,
2015-01-02,118.93,251400000.0,113.637510,-0.005768,-0.005768,-0.007305,2.989900e+10,2.856847e+10
2015-01-05,117.34,250200000.0,112.118267,-0.013369,-0.013369,-0.004773,2.935847e+10,2.805199e+10
2015-01-06,115.31,242900000.0,110.178604,-0.017300,-0.017300,-0.029177,2.800880e+10,2.676238e+10
2015-01-07,116.73,240900000.0,111.535412,0.012315,0.012315,-0.008234,2.812026e+10,2.686888e+10
2015-01-08,118.71,237600000.0,113.427301,0.016962,0.016962,-0.013699,2.820550e+10,2.695033e+10
2015-01-09,117.57,240400000.0,112.338032,-0.009603,-0.009603,0.011785,2.826383e+10,2.700606e+10
2015-01-12,117.18,238400000.0,111.965387,-0.003317,-0.003317,-0.008319,2.793571e+10,2.669255e+10
2015-01-13,117.27,236900000.0,112.051382,0.000768,0.000768,-0.006292,2.778126e+10,2.654497e+10
2015-01-14,116.80,241650000.0,111.602297,-0.004008,-0.004008,0.020051,2.822472e+10,2.696870e+10


In [119]:
def OLD_calc_mwr(df):
    ''' irr in period'''
    begin_val = 10000.
    
    end_val = df.mkt_cap[-1]/df.mkt_cap[0]*10000.
    cum_flow = (df.flow+1).cumprod()-1
    
    balance = pd.Series(index=net_flows.index)
    balance.iloc[0] = begin_val #+ net_flows.iloc[0]
    
    
    net_flows.iloc[-1] = end_val + net_flows.iloc[-1]
    irr = np.irr(net_flows)
    print(irr)
    print(net_flows)
    
    cum_ret = (1+irr)**len(net_flows)-1
    
    return cum_ret


,,close,so,daily_ret,flow,mkt_cap
date,symbol,,,,,
2017-06-28,SPY,243.49,982682112.0,0.008950,-0.003549,2.392733e+11
2017-06-29,SPY,241.35,981332096.0,-0.008789,-0.001374,2.368445e+11
2017-06-30,SPY,241.80,978982144.0,0.001865,-0.002395,2.367179e+11
2017-07-03,SPY,242.21,979182144.0,0.001696,0.000204,2.371677e+11
2017-07-05,SPY,242.77,977582144.0,0.002312,-0.001634,2.373276e+11


In [89]:

#cash_flow_series = np.array([begin_val]),net_flows.values,end_val])
#np.irr(cash_flow_series)

In [214]:
df = ibb.xs(slice('2012-01-01',None),level='date',drop_level=False)#ibb.loc[slice('2014-01-04',None)]
adj_flows = df.flow*(df.mkt_cap/df.mkt_cap.iloc[0]) #contributions in beginning of period dollars
adj_flows.iloc[0] = 1 # initial principal
basis = (adj_flows).cumsum() #cumulative contributions in beginning of period dollars
adj_ret = df.daily_ret*basis # daily returns weighted by amount of capital
adj_growth_factor = adj_ret + 1
print("MWR: {}".format(((adj_ret+1).product()-1)/basis.mean()))
print("TWR: {}".format(calc_twr(df)))


MWR: -0.18491837846508344
TWR: 1.7661715380929603


In [412]:
def calc_approx_mwr(df):
    ''' Calculate the return for the full time period normalized for amount of capital deployed.
    '''
    
    # First, calculate the invested "basis" in terms of day 0 dollars.  
    adj_flows = df.flow*(df.mkt_cap/df.mkt_cap.iloc[0]) 
    adj_flows.iloc[0] = 1 # initial principal
    basis = (adj_flows).cumsum() #cumulative contributions in beginning of period dollars
    
    # Next, convert daily returns to log basis and weight by amount of capital deployed
    # to calculate an average compound return rate (log)
    log_rets = np.log(df.daily_ret+1)
    avg_daily_log_ret = (log_rets*basis / basis.mean()).mean()
    
    # finally, convert into a simple return for the full period
    cum_ret = (avg_daily_log_ret+1)**(len(log_rets))-1
    
    return cum_ret

df = spy.loc['2012']
print("MWR: {}".format(calc_approx_mwr(df)))
print("TWR: {}".format(calc_twr(df)))

MWR: 0.12225087176730187
TWR: 0.1347410358565735


In [310]:
twr = ibb.xs('IBB',level='symbol').resample('A').apply(calc_twr)
twr.name = 'twr'

mwr = ibb.xs('IBB',level='symbol').resample('A').apply(calc_dollar_wtd_return)
mwr.name = 'mwr'
a = pd.concat([twr,mwr],axis=1)
a

,twr,mwr
date,,
2008-12-31,-0.117062,-0.126849
2009-12-31,0.151724,0.147788
2010-12-31,0.141635,0.137532
2011-12-31,0.116999,0.093576
2012-12-31,0.314998,0.266257
2013-12-31,0.654715,0.613040
2014-12-31,0.341584,0.333664
2015-12-31,0.094544,0.093242
2016-12-31,-0.198878,-0.182096


In [296]:
mwr

date
2001-12-31   -0.150529
2002-12-31   -0.207484
2003-12-31    0.259101
2004-12-31    0.121244
2005-12-31    0.028147
2006-12-31    0.145736
2007-12-31    0.020229
2008-12-31   -0.409775
2009-12-31    0.239060
2010-12-31    0.129616
2011-12-31   -0.007705
2012-12-31    0.122251
2013-12-31    0.287759
2014-12-31    0.098671
2015-12-31    0.017413
2016-12-31    0.064487
2017-12-31    0.084245
Freq: A-DEC, dtype: float64

In [231]:
(np.log((df.daily_ret+1)).mean()+1)**(len(df.daily_ret))-1

1.7646530733454409

In [256]:
df = ibb.loc['2012']#.xs(slice('2012-01-01',None),level='date',drop_level=False)
# TWR (weight all equally)
log_rets = np.log((df.daily_ret+1))
avg_daily_log_ret = log_rets.mean()
compound_growth = (avg_daily_log_ret+1)**(len(df.daily_ret))-1
print("TWR: {}".format(compound_growth))

# MWR (weight by dollar values)
log_rets = np.log(df.daily_ret+1)
avg_daily_log_ret = (log_rets*basis / basis.mean()).mean()
compound_growth = (avg_daily_log_ret+1)**(len(df.daily_ret))-1
print("MWR: {}".format(compound_growth))


TWR: 0.31479976057251835
MWR: 0.1395297599284404


In [277]:
df['log_ret'] = np.log(1+df.daily_ret)
df['adj_flow'] = df.flow*(df.mkt_cap/df.mkt_cap.iloc[0])
df['future_avg_log_ret'] = df.sort_index(ascending=False).log_ret.expanding().mean() 
df.future_avg_log_ret*df.flow


C:\Anaconda2\envs\blog\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda2\envs\blog\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda2\envs\blog\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata

date        symbol
2012-01-03  IBB       0.000008
2012-01-04  IBB       0.000008
2012-01-05  IBB       0.000049
2012-01-06  IBB       0.000000
2012-01-09  IBB      -0.000007
2012-01-10  IBB      -0.000007
2012-01-11  IBB       0.000029
2012-01-12  IBB      -0.000051
2012-01-13  IBB      -0.000006
2012-01-17  IBB       0.000000
2012-01-18  IBB      -0.000009
2012-01-19  IBB      -0.000012
2012-01-20  IBB       0.000000
2012-01-23  IBB      -0.000006
2012-01-24  IBB       0.000020
2012-01-25  IBB       0.000038
2012-01-26  IBB       0.000013
2012-01-27  IBB       0.000003
2012-01-30  IBB       0.000010
2012-01-31  IBB       0.000016
2012-02-01  IBB      -0.000005
2012-02-02  IBB      -0.000029
2012-02-03  IBB       0.000009
2012-02-06  IBB       0.000000
2012-02-07  IBB       0.000004
2012-02-08  IBB       0.000000
2012-02-09  IBB       0.000023
2012-02-10  IBB       0.000009
2012-02-13  IBB      -0.000004
2012-02-14  IBB       0.000004
                        ...   
2012-11-16  IBB     

In [261]:
np.e**(log_rets.cumsum())-1


date        symbol
2012-01-03  IBB       0.009296
2012-01-04  IBB       0.004983
2012-01-05  IBB       0.019166
2012-01-06  IBB       0.027408
2012-01-09  IBB       0.041687
2012-01-10  IBB       0.062290
2012-01-11  IBB       0.068807
2012-01-12  IBB       0.076282
2012-01-13  IBB       0.076665
2012-01-17  IBB       0.083373
2012-01-18  IBB       0.095640
2012-01-19  IBB       0.090656
2012-01-20  IBB       0.082894
2012-01-23  IBB       0.076378
2012-01-24  IBB       0.082607
2012-01-25  IBB       0.109056
2012-01-26  IBB       0.099281
2012-01-27  IBB       0.112985
2012-01-30  IBB       0.107810
2012-01-31  IBB       0.111644
2012-02-01  IBB       0.135889
2012-02-02  IBB       0.142405
2012-02-03  IBB       0.156109
2012-02-06  IBB       0.168088
2012-02-07  IBB       0.160038
2012-02-08  IBB       0.151030
2012-02-09  IBB       0.144609
2012-02-10  IBB       0.139243
2012-02-13  IBB       0.165884
2012-02-14  IBB       0.155534
                        ...   
2012-11-16  IBB     

## Summary
This post presented the concept of organizing data into a `features` dataframe and `outcome` dataframe, and then showed how simple it is to join these two dataframes together to train a model.  

True, the convention may take a few examples to get used to.  However, after trial and error, I've found this to be the most error-resistant, flexible, and high-performance way to go.

In the [next post], I will share some methods of feature engineering and feature selection.  

